# [![](https://img.shields.io/badge/Thiết%20kế-stablediffusion.vn-0075ff)](https://stablediffusion.vn) [![](https://img.shields.io/badge/Phiên%20bản-v3-0075ff)](https://stablediffusion.vn) [![](https://img.shields.io/badge/Legacy-Version-ff0000)](https://bit.ly/sdvndreamboot) [![](https://img.shields.io/badge/Group-Support-0075ff)](https://www.facebook.com/groups/stablediffusion.vn) [![](https://img.shields.io/badge/Bộ%20công%20cụ-Đầy%20đủ-0075ff)](https://stablediffusion.vn/bo-cong-cu/) [![](https://img.shields.io/discord/813085864355037235?color=blue&label=Discord&logo=Discord)](https://discord.gg/5SEtApPeyG) [![](https://img.shields.io/badge/Kohya%20Setting-Wiki-Blue)](https://github.com/bmaltais/kohya_ss/wiki/LoRA-training-parameters) [![](https://img.shields.io/badge/Khoá%20học-SD-red)](https://hungdiffusion.com/)

---
# 💡 Hướng dẫn sử dụng :


 - B1 : Chuẩn bị thư mục ảnh mẫu để train theo đúng kích thước, tải lên drive, có thể phân chia thư mục con
 - B1 : Chạy phần 1
 - B2 : Chỉnh Path để dẫn đến thư mục data đã tải và chạy phần 2
 - B3 : Chỉnh các thông số train tại mục 3 và chạy phần 3
 - B4 : Chạy mục 4 và theo dõi kết quả train
 ---

 🔻 : Lưu ý quan trọng thay đổi theo mỗi lần train

 🔸 : Giá trị tham khảo, có thể để mặc định


# 🔌 1. Cài đặt nền

In [ ]:
# @title Install
from google.colab import drive
drive.mount('/content/drive')

#root_dir
root_dir = "/content"
repo_dir = f"{root_dir}/Kohya-Colab"
training_dir = f"{root_dir}/Train_Config"
pretrained_model = f"{root_dir}/Train_model"
config_dir = f"{training_dir}/Config"
#repo_dir
accelerate_config = f"{repo_dir}/accelerate_config/config.yaml"
tools_dir = f"{repo_dir}/tools"
finetune_dir = f"{repo_dir}/finetune"

!git clone https://github.com/StableDiffusionVN/SDVN-kohya-colab-sd15 {repo_dir}

%run {repo_dir}/TrainScript.ipynb

install_sd15train()

# 📂 2. Kết nối data Train

In [ ]:
# @title
#@markdown ##<br>🔻 2.1 Đường dẫn thư mục dữ liệu trong drive
#@markdown <br>💡 `Có thể bỏ trống RegFolder`
TrainFolder = "/content/drive/MyDrive/Chasis"  # @param {type:'string'}
RegFolder = ""  # @param {type:'string'}
#@markdown <br>💡 `Weight của thư mục ảnh reg`
Prior_loss_weight = 1  # @param {type:"number"}
DataClean = False #@param {type:"boolean"}
SubFolder = True
#@markdown ##<br>🔸 2.2 Tạo caption tự động
AutoCaption = "Waifu-1.4" # @param ["none", "BLIP-Caption", "Waifu-1.4"]
Caption_Length = "Medium" # @param ['Short','Medium','Long']

# @markdown 🟡 `Thêm caption tuỳ chọn`
Custom_Caption = "" # @param {type:'string',placeholder:"Nhập từ khoá tuỳ chọn hoặc chuỗi muốn xoá khi chọn chế độ Remove"}
Remove_Caption = False #@param {type:"boolean"}
Append = False
# @markdown 🟡 `Tự động nhận tên thư mục con làm caption`
AddFolderName = False #@param {type:"boolean"}
SubFolder = True

Cap_prompt = {
    'Short':['<CAPTION>',10,30,0.8],
    'Medium':['<DETAILED_CAPTION>',10,100,0.5],
    'Long':['<MORE_DETAILED_CAPTION>',10,150,0.3]
}

Cap_extension = ".txt"
extension = Cap_extension

if DataClean == True :
    %cd {root_dir}
    clean_directory(TrainFolder)

if AutoCaption == "Waifu-1.4" :
    %cd {finetune_dir}
    config = {
        "_train_data_dir": TrainFolder,
        "batch_size": 6,
        "repo_id": "SmilingWolf/wd-v1-4-convnext-tagger-v2",
        "recursive": True,
        "remove_underscore": True,
        "general_threshold": Cap_prompt[Caption_Length][3],
        "character_threshold": 0.2,
        "caption_extension": Cap_extension,
        "max_data_loader_n_workers": 2,
        "debug": True,
        "undesired_tags": ""
    }
    final_args = f"python {finetune_dir}/tag_images_by_wd14_tagger.py {join_arg(config)}"
    print(final_args)
    !{final_args}

if AutoCaption == "BLIP-Caption" :
    %cd {finetune_dir}
    config = {
        "_train_data_dir" : TrainFolder,
        "batch_size" : 6,
        "beam_search" : True,
        "min_length" : Cap_prompt[Caption_Length][1],
        "max_length" : Cap_prompt[Caption_Length][2],
        "debug" : True,
        "caption_extension" : Cap_extension,
        "max_data_loader_n_workers" : 2,
        "recursive" : True
    }
    final_args = f"python {finetune_dir}/make_captions.py {join_arg(config)}"
    print(final_args)
    !{final_args}

if AddFolderName:
  add_forder_name(TrainFolder)
if Custom_Caption != "":
  process_dir(TrainFolder, Custom_Caption, Append, Remove_Caption)


# ⚙️ 3. Cài đặt Train

In [ ]:
#@markdown ##<br>🔻 3.1 Cài đặt chung
#@markdown <br>

Train_Option = "Lora" #@param ["Checkpoint", "Lora"]
Project_name = "Vehicle" #@param {type:"string",placeholder:"Nhập tên output"}
Output_Path = "/content/drive/MyDrive/stable_diffusion_weights/myChassis" #@param {'type':'string'}
#@markdown <br>💡 `Dán link tải hoặc link đường dẫn drive để sử dụng model bất kỳ để train`
Model_Train = "StableDiffusion_v15.ckpt"  #@param ["", "StableDiffusion_v15.ckpt", "Chilloutmix", "RealisticVision51", "PhotographyProv8", "Deliberate", "DreamShaperv8", "A-ZovyaRPGv3", "HenmixRealv4", "Fantasticv65", "XXMix_9realisticv3", "MajicmixRealv6", "ReVAnimatedv122", "RunDiffusionFXv1", "BeautifulRealAsiansv6", "LEOSAM-MoonFilm-v2","Counterfeit", "OrientalMix", "FishMix", "ThreeDelicacyv2", "GODofSIMP", "TheoldfishSpread", "Theoldfish", "NightSkyYOZORA", "GhostMixv2", "ToonYou-beta6", "ComicsArthemyv3", "MeinaMix-v11", "ManmaruMix-v2"] {allow-input: true}
Resume_Path = "" # @param {type:'string',placeholder:"Nhập tên đường dẫn thư mục State nếu muốn train tiếp từ backup"}
Save_State = False #@param {type:"boolean"}
#@markdown <br>💡 `Trường hợp train với SubFolder, đặt tên thư mục sub theo cú pháp " {repeat}_{name}" để đặt repeat cho từng thư mục`
Repeats = 24  # @param {type:"number"}
Resolution = 768  # @param {type:"slider", min:512, max:1024, step:128}
Optimizer_type = "AdamW8bit"  # @param ["AdamW", "AdamW8bit", "Lion", "SGDNesterov", "SGDNesterov8bit", "DAdaptation", "AdaFactor"]
#@markdown <br>💡 `Cài đặt sao lưu sau số vòng n`
Save_Every_N_Epochs = 1  # @param {type:"number"}
#@markdown ##<br>🔻 3.2-A | Checkpoint Setting
#@markdown <br>💡 `Sử dụng card T4, Batch size tối đa tới size 512 là 4, với size 768 là 2`
Batch_size = 2  # @param {type:"number"}
Learning_Rate = 1e-6  # @param {'type':'number'}
#@markdown <br>💡 `Số bước train tối đa`
Max_train_steps = 1000  # @param {type:"number"}
#@markdown ##<br>🔻 3.2-B | Lora Setting
#@markdown <br>
LoRA_Network_Weights = "" # @param {'type':'string',placeholder:"Nhập đường dẫn lora, nếu muốn train tiếp vào lora bất kỳ"}
#@markdown 💡 `Sử dụng card T4, Batch size tối đa tới size 512 là 6, với size 768 là 3`
Lora_Batch_size = 3  # @param {'type':'number'}
Network_dim = 64  # @param {'type':'number'}
Network_alpha = 32 # @param {'type':'number'}
Unet_Lr = 1e-4  # @param {'type':'number'}
Text_encoder_Lr = 1e-4  # @param {'type':'number'}
#@markdown <br>💡 `Số vòng train tối đa`
Epochs = 4  # @param {type:"number"}
#@markdown <br>💡 `Cài đặt Lerning rate scheduler`
Lr_scheduler = "constant"  # @param ["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup", "adafactor"] {allow-input: false}
Lr_power = 2 # @param {'type':'number'}
Comment = "creator by SD15" # @param {'type':'string'}


output_name = Project_name if Project_name != "" else "project_name"
resume = Resume_Path
save_state = Save_State
optimizer_type = Optimizer_type
save_every_n_epochs = Save_Every_N_Epochs
training_comment = Comment
dataset_repeats = Repeats
max_train_steps = Max_train_steps if Train_Option == "Checkpoint" else None
train_batch_size = Lora_Batch_size if Train_Option == "Lora" else Batch_size
network_weights = LoRA_Network_Weights
network_dim = Network_dim
network_alpha = Network_alpha
learning_rate = Learning_Rate if Train_Option == "Checkpoint" else None
text_encoder_lr = Text_encoder_Lr
unet_lr = Unet_Lr
max_train_epochs = Epochs if Train_Option == "Lora" else None
lr_scheduler = Lr_scheduler
lr_scheduler_power = Lr_power if Lr_scheduler == "polynomial" else None
lr_warmup_steps = 100 if Lr_scheduler == "constant_with_warmup" else None
lr_scheduler_num_cycles = 2 if Lr_scheduler == "cosine_with_restarts" else None
optimizer_args = "[ \"scale_parameter=False\", \"relative_step=False\", \"warmup_init=False\" ]" if  Optimizer_type == "AdaFactor" else None
output_dir = f'{Output_Path}/{"Lora" if Train_Option == "Lora" else "Model"}'
activation_word = output_name

vae_link = "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt"
vae_name = "vae.ckpt"
aria_down(vae_link,pretrained_model,vae_name)
vae = f"{pretrained_model}/{vae_name}"

file_path = f'{repo_dir}/model_lib.json'
with open(file_path, 'r') as json_file:
    modellist = json.load(json_file)

pretrained_model_name_or_path = download_lib(Model_Train,modellist,pretrained_model)

sample_prompt = f"{config_dir}/sample_prompt.txt"
config_file = f"{config_dir}/config_file.toml"
dataset_config = f"{config_dir}/dataset_config.toml"

default_config_lora = f"{repo_dir}/config/default_config_lora.json"
default_config_checkpoint = f"{repo_dir}/config/default_config_checkpoint.json"

final_config(default_config_lora if  Train_Option == "Lora" else default_config_checkpoint,config_file)

subsets = get_subsets(TrainFolder,RegFolder)

config = {
    "general": {
        "enable_bucket": True,
        "caption_extension": Cap_extension,
        "shuffle_caption": True,
        "keep_tokens": 0,
        "bucket_reso_steps": 64,
        "bucket_no_upscale": False,
    },
    "datasets": [
        {
            "resolution": Resolution,
            "min_bucket_reso": 320,
            "max_bucket_reso": 1280,
            "caption_dropout_rate": 0,
            "caption_tag_dropout_rate": 0,
            "caption_dropout_every_n_epochs": 0,
            "flip_aug": False,
            "color_aug": False,
            "face_crop_aug_range": None,
            "subsets": subsets,
        }
    ],
}

config_str = toml.dumps(config)
write_file(dataset_config, config_str)

dir = random.choice(subsets)["image_dir"]
txt_files = [f for f in os.listdir(dir) if f.endswith('.txt')]
sample = read_file(f"{dir}/{random.choice(txt_files)}")
default_prompt = "best quality --n worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry --w 512 --h 768 --l 7 --s 28 "
sample_str = f"{sample},{default_prompt}"
write_file(sample_prompt, sample_str)

print("Done")

# ⏳ 4.Train

In [ ]:
#@markdown #▶️ Theo dõi quá trình và kiểm soát chất lượng
#@markdown *Kiểm tra sample tại ```{Output_Path}/sample```*
#@markdown <br>*Có thể dừng bất cứ khi nào sample thấy ưng ý*

%cd {repo_dir}

if Train_Option == "Checkpoint" :
  train_file = "train_db.py"
else :
  train_file = "train_network.py"

accelerate_conf = {
    "config_file" : accelerate_config,
    "num_cpu_threads_per_process" : 1,
}

train_conf = {
    "sample_prompts" : sample_prompt,
    "dataset_config" : dataset_config,
    "config_file" : config_file
}

accelerate_args = join_arg(accelerate_conf)
train_args = join_arg(train_conf)
final_args = f"accelerate launch {accelerate_args} {train_file} {train_args}"
print(final_args)
!{final_args}